# Viewing TLE data

This [Jupyter](https://jupyter.org) notebook explores and analyzes the raw [Two-line Element Set (TLE)](https://en.wikipedia.org/wiki/Two-line_element_set) satellite tracking data provided for the 12/2020 VAULT technical scenario, showing what data is available and how it can be accessed and visualized from Python. The notebook also acts as a runnable application that can be put on a server to allow users to explore the raw data interactively.

In [ ]:
import os
import time, datetime, calendar
import pandas as pd
import numpy as np
import colorcet as cc
import panel as pn
import datetime as dt
import holoviews as hv
from holoviews.operation.datashader import rasterize, dynspread
import skyfield
from skyfield.api import EarthSatellite, load
from skyfield.framelib import itrs
from IPython.display import display
hv.extension('bokeh')

# Loading data

Here let's work with the raw data from 2017:

In [ ]:
basename = "tle2017"

The raw data consists of TLE records, where each record is a three-line string, with the 2nd and 3rd lines containing the data about the satellite position, heading, etc. We can use the [skyfield](https://rhodesmill.org/skyfield) library to calculate the longitude and latitude location of the satellite (projected down on the earth's surface) at the time of the TLE record entry.

In [ ]:
def modulo_lon(val):
    return (val+180) % 360 - 180

def compute_lat_lon(line1, line2):
    """Get the Lat/Lon at the TLE epoch"""
    sat = EarthSatellite(line1, line2)
    lat, lon, _ = sat.at(sat.epoch).frame_latlon(itrs)
    return lat.degrees, lon.degrees

def lat_lon_from_lines(lines):
    lons, lats = [], []
    for line1, line2 in lines:
        lat, lon = compute_lat_lon(line1,line2)
        if None not in [lon, lat]:
            lons.append(modulo_lon(lon))
            lats.append(lat)
    return np.array(lats), np.array(lons)

Parsing the strings and calculating locations can take up to an hour on a laptop for a large file like this, so we'll cache the results into a Parquet (efficient columnar file format) file for subsequent runs. 

In [ ]:
def load_data():
    
    # Read much-smaller cached Parquet file from disk if available
    path = "./data/satellite data/"
    cache_file = path+basename+".parq"
    if os.path.exists(cache_file):
        print("Reading Parquet file")
        df = pd.read_parquet(cache_file)
        return df
    
    print("Reading CSV file")
    tle = pd.read_csv(path+'Cleaned TLE/'+basename+'.csv')
    tle.head(2)
    print("\nSplitting lines")
    new_lines = [el.replace('None\n', '').split('\n')[:2] for el in tle['tle']]
        
    #display(tle.head(1))
    print("\nRaw TLE",tle.iloc[0]['tle']) # Example TLE record
    print("\nLine pair", new_lines[0])
                
    print("\nExtracting lat/lon")
    lines=2000
    lines=None # comment out for quick run
    lats, lons = lat_lon_from_lines(new_lines[:lines])
        
    print("\nProjecting lat/lon")
    eastings, northings = hv.util.transform.lon_lat_to_easting_northing(lons, lats)
        
    # Cache projected values
    df = pd.DataFrame(dict(x=eastings, y=northings))
    df.to_parquet(cache_file)
        
    return df

In [ ]:
%%time
df = pn.state.as_cached("projected", load_data)
df.tail()

# Viewing data

Using the projected coordinates, we can plot them with [Datashader](https://datashader.org) and [HoloViews](https://holoviews.org) on a web tile map.

To zoom in & interact with the plot, click the “Wheel zoom” tool in the toolbar on the side of the plot.
Click-and-drag the plot in order to look around.  As you zoom in, finer-grained detail will emerge and fill in.  Depending on the size of the dataset and your machine, this might take a second.

In [ ]:
tiles  = hv.element.tiles.ESRI().opts(alpha=0.5, bgcolor='black') 
points = rasterize(hv.Points(df)).opts(width=900, height=600, cmap=cc.kbc[64:], cnorm='eq_hist')
tiles * dynspread(points)

We can see lots of interesting structure in the locations of the satellites at the epoch time of each record. To help make sense of it, let's overlay data about the [locations of dedicated and collateral tracking radar stations](https://en.wikipedia.org/wiki/Solid_State_Phased_Array_Radar_System) in the [Space Surveillance Network](https://en.wikipedia.org/wiki/File:Space_Surveillance_Network.jpg):

In [ ]:
stations = {"Beale":       (39.136111, -121.436389),
            "Cape Cod":    (41.753333,  -70.538611),
            "Clear":       (64.290556, -149.187222),
            "Thule":       (76.567850,  -68.284214),
            "Fylingdales": (54.358889,   -0.669722),
            "Eglin":       (30.489444,  -86.542222),
            "Cavalier":    (48.731944,  -97.904444),
            "Ascension":   (-7.969444,  -14.393889),
            "Cobra Dane":  (52.712222,  174.113611)}

stations = {k:hv.util.transform.lon_lat_to_easting_northing(v[1],v[0]) for k,v in stations.items()}

In [ ]:
radar = hv.Points(list(stations.values())).opts(color='white', size=5)
names = hv.Labels({('x', 'y'): list(stations.values()), 
                    'text': list(stations.keys())}, ['x', 'y'], 'text')
names = names.opts(text_font_size='10pt', yoffset=0.5e6, text_color="white")

In [ ]:
tiles  = hv.element.tiles.ESRI().opts(alpha=0.5, bgcolor='black') 
points = rasterize(hv.Points(df)).opts(width=900, height=600, cmap=cc.kbc[64:200], cnorm='eq_hist')
tiles * dynspread(points) * radar * names

Now we can see that most of the records (at the epoch) are from when the satellites were close to one of the radar-based monitoring stations indicated, apart from those along the equator (presumably in geostationary orbit) and a few other widely distributed observations.

# App

Finally, we'll package up the above plot into a servable app using [Panel](https://panel.holoviz.org) (try `panel serve <thisfile>.ipynb`):

In [ ]:
pn.Column("# Raw TLE data", 
          "TLE records plotted at the epoch time in geo coordinates, "
          "with Space Surveillance Network radar stations overlaid",
          tiles * points * radar * names).servable();